In [1]:
import pandas as pd
import numpy as np

In [4]:
anp = pd.read_pickle('../../data/anp.pickle')
face = pd.read_pickle('../../data/face.pickle')
celebrity = pd.read_pickle('../../data/celebrity.pickle')
image_data = pd.read_pickle('../../data/image_data.pickle')
image_metrics = pd.read_pickle('../../data/image_metrics.pickle')
object_labels = pd.read_pickle('../../data/object_labels.pickle')
survey = pd.read_pickle('../../data/survey.pickle')

In [16]:
survey['insta_user_id'] = survey['insta_user_id'].astype(int)
image_data['user_id'] = image_data['user_id'].astype(int)

In [17]:
m = pd.merge(survey, image_data, how='inner', left_on='insta_user_id', right_on='user_id')
#m = pd.merge(m, anp, how='inner', on='image_id')
#m = pd.merge(m, celebrity, how='inner', on='image_id')
m = pd.merge(m, face, how='inner', on='image_id')
#m = pd.merge(m, image_metrics, how='inner', on='image_id')
#m = pd.merge(m, object_labels, how='inner', on='image_id')

In [18]:
faces = m[['user_id', 'image_id','face_emo']][(m.emo_confidence >= 85)].drop_duplicates()

In [19]:
faces.head()

,user_id,image_id,face_emo
2,619868570,578408166026890952_619868570,HAPPY
23,619868570,578437393992371598_619868570,HAPPY
26,619868570,623287028455820472_619868570,HAPPY
30,619868570,620476278796439469_619868570,HAPPY
38,619868570,676962397104623321_619868570,HAPPY


In [20]:
faces['happy_face'] = faces.face_emo.apply(lambda x: 1 if x=='HAPPY' else 0)
happy_face_df = a.groupby(['user_id','image_id']).agg({'happy_face': lambda x: x.sum()/len(x)}).reset_index()
happy_face_df = happy_face_df.groupby('user_id').agg({'happy_face':np.mean}).reset_index()

Merge the new feature with the survey

In [21]:
survey_with_new_feature = pd.merge(survey, happy_face_df, left_on='insta_user_id', right_on='user_id')

In [22]:
survey_with_new_feature.head()

,index,id,gender,born,education,employed,income,A_2,N_1,P_1,...,R,M,A,PERMA,N_EMO,P_EMO,imagecount,private_account,user_id,happy_face
0,0,920bf027f7d13dbdc7b66b3d3324903c,Male,1975,College graduate,Employed for wages,"$30,000 to $39,999",4,5,5,...,4.666667,5.000000,3.666667,4.2500,4.000000,4.333333,465.0,public,619868570,0.800000
1,1,b433b2bfe49e28d0b7c45925b53084e0,Male,1978,College graduate,Employed for wages,"$20,000 to $29,999",8,0,9,...,9.000000,8.000000,7.666667,8.2500,0.000000,9.000000,6.0,public,187920333,1.000000
2,4,f4f54676f75f47c17dc434cf68845328,Female,1990,High school graduate,Employed for wages,"$80,000 to $89,999",7,3,8,...,7.666667,7.666667,7.333333,7.5625,3.333333,8.000000,767.0,public,33420910,0.839674
3,5,a27a5fc47a59f35761705330253a58e3,Male,1997,High school graduate,Employed for wages,"$20,000 to $29,999",7,5,7,...,3.666667,6.666667,5.666667,5.3750,6.000000,6.000000,43.0,public,2143580844,0.800000
4,6,0a1002b2232a4ecbde604462f6d84bf9,Female,1993,College graduate,A student,"$20,000 to $29,999",7,5,7,...,6.000000,6.666667,7.000000,NaN,3.000000,5.666667,73.0,public,263042348,0.975000
